In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/positive-and-negative-test-cases/Labelled_Test_Cases.csv


In [2]:
df = pd.read_csv('/kaggle/input/positive-and-negative-test-cases/Labelled_Test_Cases.csv', encoding='latin1') 

In [3]:
df.shape

(3000, 5)

In [4]:
df.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Neg,A customer cannot stream a movie if they purc...,NaN,NaN,NaN
1,Neg,A database management system such as Oracle D...,NaN,NaN,NaN
2,Neg,All credit card information will be not be se...,NaN,NaN,NaN
3,Neg,Not all movies shall be streamed on demand at ...,NaN,NaN,NaN
4,Neg,An employee is unable to successfully use the...,NaN,NaN,NaN
5,Neg,Any interface between a user and the automate...,NaN,NaN,NaN
6,Neg,"Table constraints like a Primary key, Foreign ...",NaN,NaN,NaN
7,Neg,File selection dialog does not show supported ...,NaN,NaN,NaN
8,Neg,Field length shown to the user on the page and...,NaN,NaN,NaN
9,Neg,Cancel button functionality is not working in ...,NaN,NaN,NaN


In [6]:
columns_to_keep = ['v1', 'v2']
df = df[columns_to_keep]

In [7]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [8]:
df.v1.value_counts()

v1
Pos    2100
Neg     900
Name: count, dtype: int64

In [9]:
# Model Development

# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Define classifiers
classifiers = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Logistic Regression': LogisticRegression()
}

# Iterate through classifiers
for name, clf in classifiers.items():
    # Build a corpus from the 'Test Case/v2' column
    corpus = df['v2'].tolist()
# Create a Bag-of-Words model using CountVectorizer(to convert the text data into numerical features) 
    # and TfidfTransformer(commonly used to normalize and scale the term frequencies)
    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()
    classifier = clf

    # Create a pipeline with a text feature vectorizer (CountVectorizer), TfidfTransformer, and the classifier
    model = Pipeline([
        ('vectorizer', vectorizer),
        ('transformer', transformer),
        ('classifier', classifier)
    ])
 # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(corpus, df['v1'], test_size=0.3, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)

    print(f'\nResults for {name}:')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:\n', report)


Results for Multinomial Naive Bayes:
Accuracy: 0.8
Classification Report:
               precision    recall  f1-score   support

         Neg       0.89      0.43      0.58       288
         Pos       0.78      0.97      0.87       612

    accuracy                           0.80       900
   macro avg       0.83      0.70      0.72       900
weighted avg       0.82      0.80      0.78       900


Results for Decision Tree:
Accuracy: 0.9266666666666666
Classification Report:
               precision    recall  f1-score   support

         Neg       0.91      0.85      0.88       288
         Pos       0.93      0.96      0.95       612

    accuracy                           0.93       900
   macro avg       0.92      0.91      0.91       900
weighted avg       0.93      0.93      0.93       900


Results for Random Forest:
Accuracy: 0.9066666666666666
Classification Report:
               precision    recall  f1-score   support

         Neg       0.94      0.76      0.84       288

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv('/kaggle/input/positive-and-negative-test-cases/Labelled_Test_Cases.csv', encoding='ISO-8859-1')

# Check the first few rows to understand the structure
print(df.head())

# Check the first few rows to understand the structure
print(df.head())

# Clean the column names (remove any extra spaces or non-ASCII characters)
df.columns = df.columns.str.strip()

# Drop unnecessary columns (those with NaN values or irrelevant data)
df = df[['v1', 'v2']]  # Keep only the 'label' column and 'text' column
df.columns = ['label', 'text']  # Rename columns for clarity

# Check if there are any missing values in the cleaned dataset
print(df.isnull().sum())

# Map 'Pos' to 1 and 'Neg' to 0
df['label'] = df['label'].map({'Pos': 1, 'Neg': 0})

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Build a pipeline to vectorize the text data and train a Support Vector Machine (SVM) classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),  # Convert text to a bag-of-words model
    ('transformer', TfidfTransformer()),  # Optional, but helps with text weighting
    ('classifier', SVC())  # Support Vector Classifier
])

# Train the model on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Pr


    v1                                                 v2 Unnamed: 2  \
0  Neg   A customer cannot stream a movie if they purc...        NaN   
1  Neg   A database management system such as Oracle D...        NaN   
2  Neg   All credit card information will be not be se...        NaN   
3  Neg  Not all movies shall be streamed on demand at ...        NaN   
4  Neg   An employee is unable to successfully use the...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
    v1                                                 v2 Unnamed: 2  \
0  Neg   A customer cannot stream a movie if they purc...        NaN   
1  Neg   A database management system such as Oracle D...        NaN   
2  Neg   All credit card information will be not be se...        NaN   
3  Neg  Not all movies shall be streamed on demand at ...        NaN   
4  Neg   An employee is unable to successfully use 

In [3]:
# Import the necessary libraries
import pandas as pd
from sklearn.naive_bayes import MultinomialNB  # 导入Multinomial Naive Bayes
from sklearn.tree import DecisionTreeClassifier  # 导入决策树分类器
from sklearn.ensemble import RandomForestClassifier  # 导入随机森林分类器
from sklearn.svm import SVC  # 导入支持向量机分类器
from sklearn.linear_model import LogisticRegression  # 导入逻辑回归分类器
from sklearn.neighbors import KNeighborsClassifier  # 导入KNN分类器

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer  # 导入文本处理工具
from sklearn.model_selection import train_test_split  # 导入数据集划分工具
from sklearn.pipeline import Pipeline  # 导入Pipeline构建工具
from sklearn.metrics import accuracy_score, classification_report  # 导入评估指标

# Define all classifiers, including KNN
classifiers = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier()  # 增加KNN分类器
}

# Read data set
df = pd.read_csv('/kaggle/input/positive-and-negative-test-cases/Labelled_Test_Cases.csv', encoding='ISO-8859-1')

# data cleansing
df.columns = df.columns.str.strip()  # 去除列名的空格
df = df[['v1', 'v2']]  # 只保留'label'和'text'列
df.columns = ['label', 'text']  # 重命名列名
df['label'] = df['label'].map({'Pos': 1, 'Neg': 0})  # 将标签'Pos'映射为1，'Neg'映射为0

# Divide the training set and the test set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.3, random_state=42)

# Traverse all classifiers
for name, clf in classifiers.items():
    # Create a corpus of textual data
    corpus = df['text'].tolist()
    
    # Create bag models (CountVectorizer) and TF-IDF Transformers (TfidfTransformer)
    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()
    
    # Create a Pipeline that contains feature extraction and classifier
    model = Pipeline([
        ('vectorizer', vectorizer),
        ('transformer', transformer),
        ('classifier', clf)
    ])

    # training model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = model.predict(X_test)

    # evaluation model
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)

    print(f'\nResults for {name}:')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:\n', report)



Results for Multinomial Naive Bayes:
Accuracy: 0.8
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.43      0.58       288
           1       0.78      0.97      0.87       612

    accuracy                           0.80       900
   macro avg       0.83      0.70      0.72       900
weighted avg       0.82      0.80      0.78       900


Results for Decision Tree:
Accuracy: 0.9244444444444444
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.85      0.88       288
           1       0.93      0.96      0.95       612

    accuracy                           0.92       900
   macro avg       0.92      0.90      0.91       900
weighted avg       0.92      0.92      0.92       900


Results for Random Forest:
Accuracy: 0.9077777777777778
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.77      0.84       288